In [1]:
!nvidia-smi

Sun Oct  2 05:17:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Downloading

In [7]:
# Download Sample Data
!gdown https://drive.google.com/uc?id=1HnW1kWVQsncoeerCqlqelHZVpo_DkZzj

Downloading...
From: https://drive.google.com/uc?id=1HnW1kWVQsncoeerCqlqelHZVpo_DkZzj
To: /content/data.zip
100% 3.02G/3.02G [01:04<00:00, 46.4MB/s]


In [8]:
!unzip -q data.zip

# Install Dependencies

In [10]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ejmjsfeu
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ejmjsfeu
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 151 kB 68.9 MB/s 
     |████████████████████████████████| 1.4 MB 61.8 MB/s 
     |████████████████████████████████| 509 kB 67.3 MB/s 
     |████████████████████████████████| 248 kB 72.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 843 kB 62.6 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
     |████████████████████████████████| 117 kB 72.9 MB/s 
  Created whee

# Training

In [26]:
!python train.py --train-data-dict data/train_dict.pickle --valid-data-dict data/val_dict.pickle --train-images data/images_train/ --valid-images data/images_validation/ --output-dir /content/output/ --ITERS 16000 --batch-size 2

[10/02 06:04:17 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

# Inference

In [17]:
import json
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

import torch
from tqdm import tqdm
from pathlib import Path


%load_ext autoreload
%autoreload 2

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Change these paths to your model, images, and saving paths



In [19]:
# Change these paths to your model, images, and saving paths
trained_model_path = "model_final_all.pth"
images_path = "/content/data/images_validation/"
save_path = "/content/output_feats/"
CLASSES_NUM = 15

In [20]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = CLASSES_NUM  # number of classes
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [21]:
import detectron2.data.transforms as T
from detectron2.structures import ImageList
aug = T.ResizeShortestEdge(
    [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
)
input_format = cfg.INPUT.FORMAT

In [22]:
cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MAX_SIZE_TEST

(800, 1333)

In [23]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, trained_model_path)  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
model = build_model(cfg)
DetectionCheckpointer(model).load(trained_model_path)
model.eval()

GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
      )
      (1): BottleneckBlock(

In [25]:
images_names = os.listdir(images_path)
images_paths = [images_path + x for x in os.listdir(images_path)]

# Generate Visual Features for each image
for image_name in tqdm(images_names):   
    my_file = Path(save_path+str(image_name.split(".")[0])+'.json')
    # Pass if the file has been processed before
    if my_file.is_file():
        continue
    
    orig_image = cv2.imread(images_path+image_name)
    orig_height, orig_width = orig_image.shape[:2]
    image = aug.get_transform(orig_image).apply_image(orig_image)
    image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
    c, height, width = image.shape
    inputs = [{"image": image, "height": height, "width": width}]
    #run model
    image_data_dict = {}

    with torch.no_grad():
        images = model.preprocess_image(inputs) 
        features = model.backbone(images.tensor)
        proposals, _ = model.proposal_generator(images, features) 
        #### orig
        instances, _ = model.roi_heads(images, features, proposals)
        mask_features = [features[f] for f in model.roi_heads.in_features]
        mask_features = model.roi_heads.pooler(mask_features, [x.pred_boxes for x in instances])
        mask_features = model.roi_heads.res5(mask_features)
        average_pooling = nn.AvgPool2d(7)
        
        mask_features = average_pooling(mask_features)
        orig_size = mask_features.size()
        mask_features = mask_features.resize_(orig_size[0], orig_size[1])
        ###

        boxes = instances[0].pred_boxes.tensor.detach().cpu().numpy()

        normalized_boxes = [[box[0] / width, box[1] / height, box[2] / width, box[3] / height] for box in boxes]
        image_data_dict['visual_feats'] = mask_features.detach().cpu().numpy().tolist()
        image_data_dict['bboxes'] = normalized_boxes

    with open(save_path+str(image_name.split(".")[0])+'.json', 'w') as f:
       json.dump(image_data_dict, f)

    # VISUALIZE THE PREDICTIONS
    # for box in normalized_boxes:
    #     x1, y1, x2, y2 = box
    #     cv2.rectangle(orig_image, (int(x1*orig_width), int(y1*orig_height)), (int(x2*orig_width), int(y2*orig_height)), (255,0,0), 2)
    # plt.imshow(orig_image)
    # plt.show()

  1%|          | 46/5863 [00:22<47:25,  2.04it/s]


KeyboardInterrupt: ignored